In [254]:
#   패키지 임포트

In [255]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#### 1. URL 찾기

In [257]:
url = 'https://finance.naver.com/research/company_list.naver?%page=1'


#### 2. request(url))> response (HTML) : str(html)

In [259]:
response =requests.get(url)
response

<Response [200]>

In [260]:
response.text[:200]

'<!--  global include -->\n\n\t\n\t\n\t\n\t\n\t\n<html lang=\'ko\'>\n<head>\n\n\n\t\n\t\n\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t<title>종목분석 리포트 : 네이버페이 증권</title>\n\t\t\t\n\t\t\n\t\n\n\n\n\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\n'

#### 3. HTML > BeautifulSoup > DataFrame : 데이터 파싱 좀 어려움

In [262]:
dom = BeautifulSoup (response.content , 'html.parser') #content 나 text나 또이또이
type(dom) # select(css-selector), select_one()

bs4.BeautifulSoup

In [263]:
selector = '#contentarea_left > div.box_type_m > table.type_1 >tr' # 이렇게 해도 문제 x

'''

selector = '#contentarea_left> ' #문제 없음
selector = '#contentarea_left> div.box_type_m' #문제 없음
selector = '#contentarea_left> div.box_type_m> table.type_1  ' #문제 없음
selector = '#contentarea_left> div.box_type_m> table.type_1 >tbody ' #하나씩 찍어보니 tbody가 없음

'''

selector = 'table.type_1 >tr'# 필요 데이터만사용 위해 단축

elements = dom.select(selector)
len(elements)




49

In [264]:
elements[2]

<tr>
<td style="padding-left:10">
<a class="stock_item" href="/item/main.naver?code=066570" title="LG전자">LG전자</a>
</td>
<td><a href="company_read.naver?nid=77126&amp;page=1">AI데이터센터 냉각 시장 공략</a><img alt="NEW" class="ico_new" height="8" src="https://ssl.pstatic.net/imgstock/images5/ico_research_new.gif" width="8"/></td>
<td>교보증권</td>
<td class="file"><a href="https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf" target="_blank"><img align="absmiddle" alt="pdf" src="https://ssl.pstatic.net/imgstock/images5/down.gif"/></a></td>
<td class="date" style="padding-left:5px">24.09.23</td>
<td class="date">515</td>
</tr>

In [265]:
 # lg전자 ,이름, 제목, pdf, 교보증권, 링크, 날짜, 번호 7개 수집
element = elements[2]
tag = element.select('td') # elements에 있는 <td> 태그 6개를 선택
len(tag), tag

(6,
 [<td style="padding-left:10">
  <a class="stock_item" href="/item/main.naver?code=066570" title="LG전자">LG전자</a>
  </td>,
  <td><a href="company_read.naver?nid=77126&amp;page=1">AI데이터센터 냉각 시장 공략</a><img alt="NEW" class="ico_new" height="8" src="https://ssl.pstatic.net/imgstock/images5/ico_research_new.gif" width="8"/></td>,
  <td>교보증권</td>,
  <td class="file"><a href="https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf" target="_blank"><img align="absmiddle" alt="pdf" src="https://ssl.pstatic.net/imgstock/images5/down.gif"/></a></td>,
  <td class="date" style="padding-left:5px">24.09.23</td>,
  <td class="date">515</td>])

In [266]:
data = {}
data['stock_name'] = tag[0].select_one('a').text # 첫번째 <td>에서 'a' 태그를 가진 값 선택해 stock_name로 설정
# data['stock_name'] = tag[0].select('a')[0].text # select 로 하면 리스트로 나와서 첫번쨰 값을 설정 해줘야함

data['stock_link'] = tag[0].select_one('a').get('href') # 'a'태그의 href 속성값을 가져온다
data['title']=tag[1].select_one('a').text
data['title_link']=tag[1].select_one('a').get('href')


data['writer']=tag[2].text# 교보증권
data['pdf_link']=tag[3].select_one('a').get('href')
data['date']=tag[4].text # td 태그의 4번
data['pv'] = tag[5].text #td 태그의 5번
data


## 이런식으로 하나하나 수집 가능

{'stock_name': 'LG전자',
 'stock_link': '/item/main.naver?code=066570',
 'title': 'AI데이터센터 냉각 시장 공략',
 'title_link': 'company_read.naver?nid=77126&page=1',
 'writer': '교보증권',
 'pdf_link': 'https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf',
 'date': '24.09.23',
 'pv': '515'}

In [267]:
# 30개를 수집 하려면?반복문 사용


#enumerate 사용해 인덱스 값 추가

'''
for idx, element in enumerate (elements):
    print( idx, element) # elements의 tr태그로 구분
    print('-' * 100)
'''


rows = []
for idx, element in enumerate(elements):
    tag = element.select('td')
    #print(idx, len(tag))
    if len(tag) == 6: # td태그가 6개일 경우에만 수집하길 원하는 정보가 있으므로
        data = {}
        data['stock_name'] = tag[0].select_one('a').text
        data['stock_link'] = tag[0].select_one('a').get('href')
        data['title'] = tag[1].select_one('a').text
        data['title_link'] = tag[1].select_one('a').get('href')
        data['writer'] = tag[2].text
        data['pdf_link'] = tag[3].select_one('a').get('href')
        data['date'] = tag[4].text
        data['pv'] = tag[5].text
        rows.append(data)

In [268]:
df = pd.DataFrame(rows)
df.tail(2)

,stock_name,stock_link,title,title_link,writer,pdf_link,date,pv
28,동국S&C,/item/main.naver?code=100130,금리 하락으로 미국에서 훈풍이 불어온다,company_read.naver?nid=77098&page=1,iM증권,https://stock.pstatic.net/stock-research/compa...,24.09.19,1518
29,SOOP,/item/main.naver?code=067160,"경쟁사 스트리머 이적, 트래픽 유입 기대",company_read.naver?nid=77097&page=1,신한투자증권,https://stock.pstatic.net/stock-research/compa...,24.09.19,1474


#### 4. file download

In [270]:
# os 패키지 :파일 시스템 관리하는 파이썬 패키지

import os
os.listdir()

['.ipynb_checkpoints',
 '00_python.ipynb',
 '01_requests_naver_stock.ipynb',
 '02_requests_daum_exchange.ipynb',
 '03_rest_api.ipynb',
 '04_requests_zigbang.ipynb',
 '05_html.ipynb',
 '06_css_selector.ipynb',
 '07_naver_relational_keywords.ipynb',
 '08_gmarket.ipynb',
 '08_naver_stock_report.ipynb',
 '09_selenium.ipynb',
 '10_xpath.ipynb',
 '11_iterator_generator.ipynb',
 '12_scrapy.ipynb']

In [271]:
path = 'reports'
# 디렉토리, 파일 존재 여부 확인
os.path.exists(path) # 파일 존재 x



False

In [272]:
# 디렉토리가있는데 생성하려고 하면 에러 발생함

if not os.path.exists(path):# 디겕토리 없으면 true
    os.makedirs(path)

In [273]:
os.path.exists(path)

True

In [274]:
os.listdir()[-2:]

['12_scrapy.ipynb', 'reports']

In [275]:
# 1. url

In [276]:
title=df.loc[0,'title']
pdf_link=df.loc[0,'pdf_link']
title,pdf_link

('AI데이터센터 냉각 시장 공략',
 'https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf')

In [277]:
# 2. request (url), response (pdf)  << pdf를 다운로드 받아옴

In [278]:
response = requests.get(pdf_link)
response

<Response [200]>

In [279]:
# 3. pdf > 저장 파싱 이런거 안하고 save (/reports)

In [280]:
filename= f'{path}/{title}.pdf' #path로 경로, title.pdf로 이름 설정
print(filename)



with open(filename ,'wb') as file:# 파일 저장할때 with사용 wb는 write binary 2진형태로 저장해주겠다~~~
# wt 하면 text 형태로 저장
#wb image, 영상파일 등등 모두 이진법으로 저장

    file.write(response.content)

reports/AI데이터센터 냉각 시장 공략.pdf


In [281]:
os.listdir('reports')

['AI데이터센터 냉각 시장 공략.pdf']

In [282]:
# pickle  : RAM을 메모리(SSD)에 저장하거나 SSD를 RAM으로 가져올떄 사용

In [285]:
import shutil #쉘스크립트 유틸 os에 기능 추가 디렉토리 삭제 가능

shutil.rmtree(path) # 삭제 통한 초기화

os.makedirs(path)

In [287]:
os.listdir(path)

[]

In [295]:
for idx,row in df.iterrows():
    print(idx, row['title'], row['pdf_link'])
    title,pdf_link = row['title'] , row['pdf_link']
    response = requests.get(pdf_link)
    filename= f'{path}/{title}.pdf' #path로 경로, title.pdf로 이름 설정


    with open(filename ,'wb') as file:# 파일 저장할때 with사용 wb는 write binary 2진형태로 저장해주겠다~~~
        file.write(response.content)

0 AI데이터센터 냉각 시장 공략 https://stock.pstatic.net/stock-research/company/34/20240923_company_481199000.pdf
1 설계와 시공능력으로 액침냉각 사업 확대 https://stock.pstatic.net/stock-research/company/34/20240923_company_279682000.pdf
2 액침냉각, 기술적 강점을 확보해 나가는 중 https://stock.pstatic.net/stock-research/company/62/20240923_company_647832000.pdf
3 3Q24 Preview: 국내 수주 강세 VS 중국 부진 .. https://stock.pstatic.net/stock-research/company/57/20240923_company_621805000.pdf
4 Re-rating 구간 돌입 https://stock.pstatic.net/stock-research/company/63/20240923_company_747255000.pdf
5 높아지는 Peak sales https://stock.pstatic.net/stock-research/company/29/20240923_company_582032000.pdf
6 Metsera, 너는 계획이 다 있구나 https://stock.pstatic.net/stock-research/company/39/20240923_company_387902000.pdf
7 의대 열풍, 나만 믿어 https://stock.pstatic.net/stock-research/company/21/20240923_company_352867000.pdf
8 속도가 느려도, 방향성은 맞다 https://stock.pstatic.net/stock-research/company/39/20240923_company_584932000.pdf
9 빅파마들의 RPT 방향, 우리도 간다 https://stock.pstatic.net/stoc

In [297]:
os.listdir(path)

['2024년 3분기 부진 터널 통과해야!.pdf',
 '3Q24 Preview',
 '3Q24 일시적인 실적 둔화,  분위기 반전이 필...pdf',
 '3분기는 비수기, 그러나   주주 가치 환원에 ...pdf',
 'AI데이터센터 냉각 시장 공략.pdf',
 'Metsera, 너는 계획이 다 있구나.pdf',
 'Moment of Truth.pdf',
 'Re-rating 구간 돌입.pdf',
 '가치가 높아지는 F박스와 K패션 해외 진출 수...pdf',
 '경쟁력 있는 CDMO, 여기에도 있다.pdf',
 '경쟁사 스트리머 이적, 트래픽 유입 기대.pdf',
 '경증 아토피 치료의 글로벌  시장 판도를 바...pdf',
 '금리 하락으로 미국에서 훈풍이 불어온다.pdf',
 '길어지고 있는 기다림.pdf',
 '높아지는 Peak sales.pdf',
 '롯데렌탈의 쏘카 지분 추가취득 당분간 중단.pdf',
 '미국 타워 판가도 인상.pdf',
 '빅파마들의 RPT 방향, 우리도 간다.pdf',
 '설계와 시공능력으로 액침냉각 사업 확대.pdf',
 '속도가 느려도, 방향성은 맞다.pdf',
 '액침냉각, 기술적 강점을 확보해 나가는 중.pdf',
 '엔지니어링 플라스틱 소재 개발 및 고도화로 ...pdf',
 '엘사.pdf',
 '의대 열풍, 나만 믿어.pdf',
 '이튼의 견고한 성장 파트너.pdf',
 '지속가능한 성장.pdf',
 '코스닥 방사성의약품 기업으로 진입 .pdf',
 '콥데이 후기',
 '확대되는 TROP2 ADC 치료제 시장.pdf',
 '환율 모멘텀 발생 예상. 자사주 추가 매입 가...pdf']